In [192]:
import pandas as pd
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [193]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

chrome_options = Options()
chrome_options.add_argument('--headless')  # Run in background
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-notifications')

driver = webdriver.Chrome(options=chrome_options)

url = "https://www.fundsexplorer.com.br/ranking"
driver.get(url)
time.sleep(5)

html = driver.page_source
df = pd.read_html(html)[0]

driver.quit()

print("Data successfully scraped!")
print(df.head())

/tmp/ipykernel_12913/2965704421.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html)[0]


Data successfully scraped!
   Fundos       Setor Liquidez Diária (R$)  P/VP Dividend Yield  \
0  AAGR11  Indefinido             5.848,90   NaN         1,03 %   
1  AAZQ11  Indefinido           654.462,24  70.0         1,35 %   
2  ABCP11   Shoppings            60.839,95  70.0         0,90 %   
3  AFHI11      Papéis         1.111.830,95  94.0         1,07 %   
4  AGRX11      Outros           383.579,90   NaN         0,81 %   

  Variação Preço Patrimônio Líquido Num. Cotistas  
0        -1,50 %                NaN             0  
1         4,31 %     208.745.497,59        29.829  
2         2,18 %     503.496.718,25        15.558  
3        -5,29 %     432.224.228,96        42.539  
4       -20,22 %                NaN             0  


In [194]:
df = pd.read_csv('/home/wvmwill/enviroment/analytics-fii/funds_explorer_data.csv')

df.head()

sectors = df['Setor'].unique()
contagem = df['Setor'].value_counts()
print(f"Quantidade de Setores: {len((sectors))}")
print(contagem)

Quantidade de Setores: 23
Setor
Indefinido                          116
Papéis                               94
Lajes Corporativas                   60
Misto                                44
Imóveis Industriais e Logísticos     44
Fundo de Fundos                      35
Fundo de Desenvolvimento             35
Shoppings                            29
Imóveis Residenciais                 10
Imóveis Comerciais - Outros           7
Hospitalar                            6
Educacional                           5
Varejo                                5
Agências de Bancos                    5
Outros                                4
Serviços Financeiros Diversos         4
Hotéis                                4
Incorporações                         2
Logística                             2
Fiagro                                1
Agricultura                           1
Tecidos. Vestuário e Calçados         1
Name: count, dtype: int64


In [195]:
df.isna().sum()

Fundos                   0
Setor                    1
Liquidez Diária (R$)    47
P/VP                    71
Dividend Yield           3
Variação Preço           3
Patrimônio Líquido      46
Num. Cotistas            0
dtype: int64

In [196]:
data_categoricals = ['Fundos', 'Setor']
idx = df[df['Setor'].isna()].index

In [197]:
df.drop(idx, inplace=True)

In [198]:
df[data_categoricals].isna().sum()


df[data_categoricals] = df[data_categoricals].astype('category')

df.dtypes

Fundos                  category
Setor                   category
Liquidez Diária (R$)      object
P/VP                     float64
Dividend Yield            object
Variação Preço            object
Patrimônio Líquido        object
Num. Cotistas             object
dtype: object

In [199]:
col_floats = list(df.iloc[:,2:-1].columns)

In [200]:
df[col_floats] = df[col_floats].fillna(value=0)
df[col_floats].head()

,Liquidez Diária (R$),P/VP,Dividend Yield,Variação Preço,Patrimônio Líquido
0,"5.848,90",0.0,"1,03 %","-1,50 %",0
1,"654.462,24",70.0,"1,35 %","4,31 %","208.745.497,59"
2,"60.839,95",70.0,"0,90 %","2,18 %","503.496.718,25"
3,"1.111.830,95",94.0,"0,00 %","0,00 %","432.224.228,96"
4,"383.579,90",0.0,"0,81 %","-20,22 %",0


In [201]:
df[col_floats].isna().sum()

Liquidez Diária (R$)    0
P/VP                    0
Dividend Yield          0
Variação Preço          0
Patrimônio Líquido      0
dtype: int64

In [202]:
df[col_floats]

,Liquidez Diária (R$),P/VP,Dividend Yield,Variação Preço,Patrimônio Líquido
0,"5.848,90",0.0,"1,03 %","-1,50 %",0
1,"654.462,24",70.0,"1,35 %","4,31 %","208.745.497,59"
2,"60.839,95",70.0,"0,90 %","2,18 %","503.496.718,25"
3,"1.111.830,95",94.0,"0,00 %","0,00 %","432.224.228,96"
4,"383.579,90",0.0,"0,81 %","-20,22 %",0
...,...,...,...,...,...
510,0,0.0,"0,00 %","0,00 %",0
511,"1.353,76",140.0,"0,54 %","0,70 %","42.783.877,65"
512,277,103.0,"1,24 %","16,56 %","31.110.512,89"
513,"204.552,71",64.0,"1,11 %","-1,39 %","154.014.555,33"


In [203]:
df['Liquidez Diária (R$)'] = (df['Liquidez Diária (R$)']
    .apply(lambda x: str(x).replace('.', '').replace(',', '.'))
    .astype(float))

In [204]:
df['Dividend Yield'] = df['Dividend Yield'].apply(lambda x: str(x).replace('%', '').replace(',','.')).astype(float)
df['Dividend Yield'] = df['Dividend Yield'].div(100)
df.head()

,Fundos,Setor,Liquidez Diária (R$),P/VP,Dividend Yield,Variação Preço,Patrimônio Líquido,Num. Cotistas
0,AAGR11,Indefinido,5848.90,0.0,0.0103,"-1,50 %",0,0
1,AAZQ11,Indefinido,654462.24,70.0,0.0135,"4,31 %","208.745.497,59",29.829
2,ABCP11,Shoppings,60839.95,70.0,0.0090,"2,18 %","503.496.718,25",15.558
3,AFHI11,Papéis,1111830.95,94.0,0.0000,"0,00 %","432.224.228,96",42.539
4,AGRX11,Outros,383579.90,0.0,0.0081,"-20,22 %",0,0


In [205]:
df['Variação Preço'] = df['Variação Preço'].apply(lambda x: str(x).replace('%', '').replace(',','.')).astype(float)
df['Variação Preço'] = df['Variação Preço'].div(100)

In [206]:
df['Num. Cotistas'] = df['Num. Cotistas'].apply(lambda x: str(x).replace('.', '')).astype(int)

In [207]:
df['Patrimônio Líquido'] = (df['Patrimônio Líquido']
    .apply(lambda x: str(x).replace('.', '').replace(',', '.'))
    .astype(float))

In [210]:
df.dtypes

df.to_csv('fiis.csv')